# EEG - Flow

## X. Check all of the annotated signal, in search of wrong annotations labels

Step 1: 
Going through all files and look for wrong labels

Step 2: 
Selectively rename labels where needed

Last edit: 19.10.2023 15:50

# Step 1: Going through all files and look for wrong labels

In [1]:
import glob
from datetime import datetime
import time

from mne.io import read_raw_fif

In [2]:
""" Finds all step3 .fif files in derivatives and store them in list """

path = f"L:\EEG_Flow_data\derivatives\\"
step3_files = glob.glob(path + "/**/*_step3_with-bads_raw.fif", recursive = True)

In [3]:
def ft_check_label(fname):
    """ This checks the labels for the annotations.
        Prints ok then return None if things are as expected (ie. bad_)
        else, print the labels that were used and return the fname """
    raw = read_raw_fif(fname, verbose = "WARNING")

    key_info = fname.split("\\")[4].split("_")
    participant = key_info[0][4:]
    task = key_info[1].split("-")[1]
    run = key_info[2].split("-")[1]

    current_annot = set(raw.annotations.description)
    print(f"{participant}, {task}, {run}")

    result = ""
    if set(current_annot) == {'bad_', 'standard', 'novel', 'target'}:
        print("- ok")
        result = "ok"
    elif set(current_annot) == {'standard', 'novel', 'target'}:
        print("- no bad_")
        result = "no bad"
    else:
        print(current_annot)
        result = "mislabel"
    print("-"*80)
    return(result, fname)

In [4]:
%%time
""" Feeds all fname to ft_check_label then stores problematic fname in list """

current_time = time.strftime("%H:%M:%S")
print("Start time", current_time)

files_to_amend = []
files_ok = []
no_bad = []

for file in step3_files:
    #print(file)
    result, fname = ft_check_label(file)
    if result == "ok":
        files_ok.append(fname)
    elif result == "no bad":
        no_bad.append(fname)
    else:
        files_to_amend.append(fname)

Start time 15:44:35
P01-G1, oddball, 1
- ok
--------------------------------------------------------------------------------
P01-G1, oddball, 2
- ok
--------------------------------------------------------------------------------
P01-G1, UT, 1
- ok
--------------------------------------------------------------------------------
P01-G1, UT, 2
- ok
--------------------------------------------------------------------------------
P02-G2, oddball, 1
- ok
--------------------------------------------------------------------------------
P02-G2, oddball, 2
- ok
--------------------------------------------------------------------------------
P02-G2, UT, 1
- ok
--------------------------------------------------------------------------------
P02-G2, UT, 2
- ok
--------------------------------------------------------------------------------
P03-G1, oddball, 1
- ok
--------------------------------------------------------------------------------
P03-G1, UT, 1
- ok
------------------------------------

In [5]:
""" Summary of files to ammend """

print("Files to ammend\n")
for file in files_to_amend:
    print(file.split("\\")[4])

Files to ammend



In [6]:
""" Summary of files with no additional annotations """
print("No additional annotations\n")
for file in no_bad:
    print(file.split("\\")[4])

No additional annotations

sub-P05-G2_task-oddball_run-2
sub-P15-G4_task-oddball_run-2
sub-P46-G3_task-oddball_run-1


In [7]:
""" Write output into a log """
timestampStr = datetime.now().strftime("%Y-%m-%d__%H-%M")

with open(r'checks/logs_labels_'+timestampStr+'.txt', 'w') as fp:
    fp.write(timestampStr)

    fp.write("\n\n")

    fp.write("Files to amend\n")
    for file in files_to_amend:
        # write each item on a new line
        fp.write("%s\n" % file.split("\\")[4])

    fp.write("\n")

    fp.write("Files with no bad_\n")
    for file in no_bad:
        # write each item on a new line
        fp.write("%s\n" % file.split("\\")[4])

    fp.write("\n")

    fp.write("Files that are ok\n")
    for file in files_ok:
        # write each item on a new line
        fp.write("%s\n" % file.split("\\")[4])
    print('Done')

Done


# Step 2: Selectively rename labels where needed

In [ ]:
""" Enter the info of the files you want to update """

PG = "P48-G8"
task = "UT"
run = 2

In [ ]:
fname = f"L:\EEG_Flow_data\derivatives\sub-{PG}\sub-{PG}_task-{task}_run-{run}\sub-{PG}_task-{task}_run-{run}_step3_with-bads_raw.fif"
raw = read_raw_fif(fname)
set(raw.annotations.description)

In [ ]:
""" raw.annotations.rename({"XXX":"bad_"}), 
first element is the wrongly labeled bad, 2nd is the correct label """

raw.annotations.rename({"badswronglabel_":"bad_"})

In [ ]:
""" check and save in temp (remember to rename the file and remove the old one)"""
set(raw.annotations.description)

In [ ]:
fname2 = f"L:\EEG_Flow_data\derivatives\sub-{PG}\sub-{PG}_task-{task}_run-{run}\sub-{PG}_task-{task}_run-{run}_step3_with-bads_bis_raw.fif"
raw.save(fname2, overwrite=False)